# Final Project - Link Prediction

For the final project, I identified key algorithms for generating social netowrks and also predicted edges using machine learning. 

In Part 1, I identified randomly generated NetworkX graphs based on their degree distributions. 

In Part 2, I evaluated an employee social network to determine which employees received a management position salary. The final task involved creating a machine learning classifier to find the probability that coworkers will form a future connection together.

---
## Part 1 - Random Graph Identification

**For part 1, evaluate the 5 networkx graphs available in the list, `P1_Graphs`.**

Each of these graphs were generated by **one of three possible algorithms**:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

### Analyze each of the 5 graphs and determine which of the three algorithms generated each graph.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

In [3]:
def degree_dist(G):
    
    # Degree values for graph
    degrees = G.degree().values()
    sorted_deg = sorted(set(degrees))
    
    dist = [list(degrees).count(i) / float(nx.number_of_nodes(G)) for i in sorted_deg]
    return dist

# Degree distribution length. Use for assessing preferential attachment:
dist = []
# Note: Small-world networks tend to have 1) high clustering coefficients and 2) low average shortest path values
for G in P1_Graphs:
    print("Degree Distribution Size: ", len(degree_dist(G)), 
          "Avg. Clustering Coefficient: ", nx.average_clustering(G),
          "Avg. Shortest Path Length: ", nx.average_shortest_path_length(G))
    dist.append(len(degree_dist(G)))
    
# For all graphs..
avg_dist = np.mean(dist)
print("Average degree distribution size: ", avg_dist)

Degree Distribution Size:  29 Avg. Clustering Coefficient:  0.03167539146454044 Avg. Shortest Path Length:  4.099161161161161
Degree Distribution Size:  7 Avg. Clustering Coefficient:  0.5642419635919628 Avg. Shortest Path Length:  5.089871871871872
Degree Distribution Size:  5 Avg. Clustering Coefficient:  0.4018222222222227 Avg. Shortest Path Length:  9.378702269692925
Degree Distribution Size:  37 Avg. Clustering Coefficient:  0.03780379975223251 Avg. Shortest Path Length:  3.1048046283934134
Degree Distribution Size:  9 Avg. Clustering Coefficient:  0.0033037037037037037 Avg. Shortest Path Length:  5.0785509568313305
Average degree distribution size:  17.4


In [4]:
def graph_identification():


    # List of possible algorithms for generating each graph
    algos = []
     
    for G in P1_Graphs:
        # Degree distribution histogram
        hist = degree_dist(G)
        # Average clustering coefficient for all 5 random graph
        clustering = nx.average_clustering(G)
        
        if len(hist) > avg_dist:
            algos.append('PA')
        ## Note: Small-world networks are more likely to rewire with a low clustering coefficient
        elif clustering < 0.1:
            algos.append('SW_H')
        else: 
            algos.append('SW_L')
        
    return algos

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For part 2, **analyze a company's email network**, where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

<br>
*Helpful Hints*:

The network contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

### Part 2A - Salary Prediction 

** Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.**

<br>
To accomplish this, create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.

Let predictions represent the probability that the corresponding employee is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [6]:
from sklearn.linear_model import LogisticRegression

# Create training and testing dataframes:
def create_df(dictionary):
    
    # Training/Testing dataframe of management salary
    df = pd.DataFrame.from_dict(dictionary, orient='index').rename(columns={0: "Target"})
    
    # Centrality Measures
    ## Note: Centrality measures must be calculated using the entire network. Once calculated, we can streamline via merge.
    centrality = [nx.degree_centrality(G), nx.closeness_centrality(G), nx.betweenness_centrality(G)]
    centrality_df = pd.DataFrame(centrality).T.rename(columns={0: "Degree", 1: "Closeness", 2: "Betweeness"})
    
    # Merge centrality measures with the training/testing dataframe
    df = centrality_df.merge(df, how='inner', left_index=True, right_index=True)
    return df


# Task: Target Value Prediction
def salary_predictions():

    # A) Create training and testing features for target (ManagementSalary)
    salary = nx.get_node_attributes(G, "ManagementSalary")

    ## Training: Contains only existing data. Denotes if employee receives management salary (0 is no, 1 is yes)
    train_dict = {k:v for k,v in salary.items() if np.isfinite(v)}
    train = create_df(train_dict)

    ## Testing: Contains missing data. Predict probability that employee receives management salary
    test_dict = {k:v for k,v in salary.items() if np.isnan(v)}
    test = create_df(test_dict)

    
    # B) Classifier.
    final = LogisticRegression(random_state=0).fit(train.iloc[:,:-1], train.iloc[:,-1])
    
    ## Make predictions on target using the test set
    y_hat = pd.Series(final.predict_proba(test.iloc[:,:-1])[:,1], index=test.index, dtype='float64')
    return y_hat

salary_predictions().head()
print('Final model - AUC Score: 0.9257 (according to Coursera\'s grader)')

Final model - AUC Score: 0.9257 (according to Coursera's grader)


### Part 2B - New Connections Prediction

The last part of the assignment involves **predicting future connections between employees** of the network. 

The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

### Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, create a matrix of features for the edges found in `future_connections` using networkx. Then, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.

Let predictions represent the probability of the corresponding edge being a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

# Task: Link Prediction
def new_connections_predictions():

    # A) Find features for target (Future_Connection)
    ind = future_connections.index

    ## Apply one node functions to employee pairs:
    one_node = [nx.preferential_attachment(G, ind), nx.jaccard_coefficient(G, ind), nx.resource_allocation_index(G, ind), 
    nx.adamic_adar_index(G, ind)]
    names = ['Pref_Attachment', 'Jaccard', 'Resource_Alloc', 'Adamic_Adar']

    for fun in range(len(one_node)):
        future_connections[names[fun]] = [i[2] for i in one_node[fun]]

    ## Apply two node function to employee pairs:
    future_connections['Comm_Neighbors'] = ind.map(lambda e: len(list(nx.common_neighbors(G, e[0], e[1]))))


    # B) Create training and testing data, for node pair without onnection (index)
    target = future_connections['Future Connection']

    ## Training: Contains only existing data. Denotes if pair will have future connection(0 is no, 1 is yes)
    train =  future_connections[target.notnull()]
    ## Testing: Contains missing data. Predict probability that pair will be a future connection
    test = future_connections[target.isnull()]

    
    # C) Classifier
    final = GradientBoostingClassifier(random_state=0).fit(train.iloc[:, 1:len(train)], train.iloc[:, 0])

    ## Make predictions on target using the test set
    y_hat = pd.Series(final.predict_proba(test.iloc[:, 1:len(test)])[:,1], index=test.index, dtype='float64')
    
    return y_hat

new_connections_predictions()
print('Final model - AUC Score: 0.9102 (according to Coursera\'s grader)')

Final model - AUC Score: 0.9102 (according to Coursera's grader)
